# Example: Solving Ordinary Differential Equations

In this notebook we will use Python to solve differential equations numerically.

In [ ]:
// Import the required modules
#r "nuget: Plotly.NET.Interactive, 3.0.2"
#r "nuget: FSharp.Stats"
#r "nuget: FSOde"

// ... and open the required modules
open FsODE
open Plotly.NET
open FSharp.Stats

Installed Packages FSharp.Stats, 0.4.7 FSOde, 0.0.1 Plotly.NET.Interactive, 3.0.2

Loading extensions from `Plotly.NET.Interactive.dll`

In [ ]:
// First set the model context that remembers the solver method and it´s optiony 
let modelContext = //OdeContext()
    OdeSolverMethod.RK546M //RK547M()
    |> OdeContext

modelContext.SetStepSize(0.01)

In [ ]:


// Parameters
let beta = 100.
let gamma = 1.

// Dynamics
let t = [|0. .. 0.01 .. 6.|]
let x (t': float) = beta / gamma * (1. - Math.Exp((-gamma) * t'))

// Mark the response time (when we get to level 1-1/e)
let t0 = 1. / gamma
let x0 = beta / gamma * (1. - Math.Exp(-1.))
[
    Array.zip t (t |> Array.map x)
    |> Chart.Line
    [t0,x0]
    |> Chart.Point
]
|> Chart.combine

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
open FSharp.Stats

// Parameters
let beta = 100.

// Dynamics
let t = [|0. .. 0.01 .. 6.|]
let x (g: float) (t': float) = 
            beta / g * (1. - Math.Exp((-g) * t'))

// Mark the response time (when we get to level 1-1/e)
let t0 (g: float) = 1. / g
let x0 (g: float) = beta / g * (1. - Math.Exp(-1.))

[
    Array.zip t (t |> Array.map (x 1.))
    |> Chart.Line
    [t0 1.,x0 1.]
    |> Chart.Point
    Array.zip t (t |> Array.map (x 2.))
    |> Chart.Line
    [t0 2.,x0 2.]
    |> Chart.Point
    Array.zip t (t |> Array.map (x 3.))
    |> Chart.Line
    [t0 3.,x0 3.]
    |> Chart.Point
]
|> Chart.combine


<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
[
    Array.zip t (t |> Array.map (fun j -> (x 1. j) / 100.))
    |> Chart.Line
    [t0 1,x0 1. / 100.]
    |> Chart.Point
    Array.zip t (t |> Array.map (fun j -> (x 2. j) / 50.))
    |> Chart.Line
    [t0 2.,x0 2. / 50.]
    |> Chart.Point
    Array.zip t (t |> Array.map (fun j -> (x 3. j) / 33.3))
    |> Chart.Line
    [t0 3.,x0 3. / 33.3]
    |> Chart.Point
]
|> Chart.combine

<!-- Plotly chart will be drawn inside this DIV -->

## Dynamical equations for the repressilator

To analyze the repressilator, we will write down our usual differential equations. For simplicity, we will assume symmetry among the species (this will not be true in the real system) and, initially, we will consider only protein dynamics, ignoring mRNA. Later on, we will ask how including mRNA modifies the conclusions.

dx1dtdx2dtdx3dt=β1+(x3/k)n−γx1,=β1+(x1/k)n−γx2,=β1+(x2/k)n−γx3.

In dimensionless units, this is

dxidt=β1+xnj−xi, with i,j pairs (1,3),(2,1),(3,2).

Fixed point

To find the fixed point of the repressilator, we solve for xi
with x˙i=0∀i

. We get that

x1x2x3=β1+xn3,=β1+xn1,=β1+xn2.

We can substitute the expression for x3
into that for x1

to get

x1=β1+(β1+xn2)n.

We can then substitute the expression for for x2

to get

x1=β1+⎛⎝⎜⎜⎜⎜β1+(β1+xn1)n⎞⎠⎟⎟⎟⎟n.

This looks like a gnarly expression, but we can write it conveniently as a composition of functions. Specifically,

x1=f(f(f(x1)))≡fff(x1),

where
f(x)=β1+xn.

By symmetry, this relation holds for repressors 2 and 3 as well, so we have

xi=fff(xi).

Writing the relationship for the fixed point with a composition of functions is useful because we can easily compute the derivatives of the composite function using the chain rule.

(ff)′(x)(fff)′(x)=f′(f(x))⋅f′(x),=f′(ff(x))⋅(ff)′(x)=f′(f(f(x)))⋅f′(f(x))⋅f′(x).

Now, since f(x)
is monotonically decreasing, f′(x)<0, and also f′(f(x))<0. This means that ff′(x)>0, so ff(x) is monotonically increasing. Now, f′(ff(x))<0, since f′(anything monotonically increasing)<0. This means that fff(x) is monotonically decreasing. Since xi is increasing, there is a single fixed point with x=fff(x). This is more clear if we look at a plot.

In [ ]:
// Parameters
let beta, n = 3., 2.

// f(x)
let f x = beta / (1. + x**n)

// Make composition of functions
let xx = [|0. .. 0.01 .. 3.|]
let fff = xx |> Array.map (fun x -> f(f(f x)))

[
    Chart.Line (xx,xx)
    Chart.Line (xx,fff)
]
|> Chart.combine

<!-- Plotly chart will be drawn inside this DIV -->

Because the time derivative of x1, x2 and x3

all vanish at the fixed point, and we have shown that the fixed point is unique, we have

x1=x2=x3≡x0=β1+xn0,

or

β=x0(1+xn0).

Because we have a single fixed point, we cannot have multistability in the repressilator. So what happens at this fixed point? To answer this question, we turn to linear stability analysis.

## Solving for the fixed point

We have shown that there is a single fixed point, x1=x2=x3≡x0

with

β=x0(1+xn0).

Getting an analytical solution of this equation for x0
is usually impossible. We therefore seek a numerical method for finding x0. This is an example of a root finding problem. It can be cast into a problem of finding f(x)=0 for some function f(x). Do not confuse this f(x) with that defined in the previous sections to define the right hand side of the dynamical equations; we are using f(x) here to be an arbitrary function. In the present case, f(x)=β−x0(1+xn0)

.

There are many algorithms for finding roots of functions. We will explore algorithms for doing so for the more general multidimensional case in future lessons. For now, we seek a scalar x0
. In this case, we know a lot about the fixed point. We know that it exists and is unique. We also know that it lies between x0=0 (where f(0)=β>0) and x0=β (since f(β)<0). When we have bounds and guarantees of uniqueness for a scalar root, we can use a bisection method to find the root. The benefit of the bisection method is that it is guaranteed to find the root of the function f(x) on an interval [a,b], provided f(x) is continuous and f(a) and f(b) have opposite sign, which is the case here. Brent’s method also has this guarantee, but is more efficient that using bisection. Brent’s method is available in the scipy.optimize.brentq() function. It takes as an argument the function f(x) whose root is to be found, and the left and right bounds for the root. We can write a function to find the fixed point for given β and n.

In [ ]:
let fixedPoint beta (n: float) =
    Optimization.Bisection.tryFindRoot
        (fun x -> beta - x * (1. + x**n))
        0.0001
        0.
        beta
        100

let betaArr = [|0. .. 0.5 .. 100.|]

[|2.;4.;8.|]
|> Array.map (fun n ->
    Chart.Line
        (
            Array.zip 
                betaArr
                (betaArr |> Array.map (fun beta -> fixedPoint beta n))
            |> Array.filter (fun (x,y) -> y.IsSome)
            |> Array.map (fun (x,y) -> x,y.Value)
        )

)
|> Chart.combine

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let repressilator_rhs (x1,x2,x3) t beta n =
    [|
        beta / (1. + x3 ** n) - x1
        beta / (1. + x1 ** n) - x2
        beta / (1. + x2 ** n) - x3
    |]

// Define a function which calculates the derivative
let dP_dt : Model = 
    fun P t ->
        let x1 = P[0]
        let x2 = P[1]
        let x3 = P[2]
        (repressilator_rhs (x1,x2,x3) t 10. 3.)

let P0 = [|1.; 1.; 1.2|]

let Ps = 
    modelContext.OdeInt(0.,P0,dP_dt)
    |> SolPoints.take 4000
    |> SolPoints.memorize 
    
let x1 = Ps |> SolPoints.toPoints 1
let x2 = Ps |> SolPoints.toPoints 2
let x3 = Ps |> SolPoints.toPoints 3

[
    Chart.Line (x1)
    Chart.Line (x2)
    Chart.Line (x3)
]
|> Chart.combine
|> Chart.withSize(1200,600)

<!-- Plotly chart will be drawn inside this DIV -->